In [2]:


import requests
import socket 
import uuid 

# python code to Check network connection
def web_site_online(url='http://www.google.com/', timeout=5):
    try:
        req = requests.get(url, timeout=timeout)
        # HTTP errors are not raised by default, this statement does that
        req.raise_for_status()
        print("Internet connection available.")
        return True
    except requests.HTTPError as e:
        print("Checking internet connection failed, status code {0}.".format(
        e.response.status_code))
    except requests.ConnectionError:
        print("No internet connection available.")
    return False

#function call
web_site_online()

# Python code to display hostname and host ip

def get_Host_name_IP(): 
    try: 
        host_name = socket.gethostname() 
        host_ip = socket.gethostbyname(host_name) 
        print("Hostname :  ",host_name) 
        print("IP : ",host_ip) 
    except: 
        print("Unable to get Hostname and IP")
        
 #function call
get_Host_name_IP()

# Python code to print MAC 
 
print ("The MAC address in formatted way is : ", end="") 
print (':'.join(['{:02x}'.format((uuid.getnode() >> ele) & 0xff) 
for ele in range(0,8*6,8)][::-1])) 

        


Internet connection available.
Hostname :   master
IP :  192.168.7.131
The MAC address in formatted way is : 94:c6:91:31:c7:5c


In [1]:
import os
import socket    
import multiprocessing
import subprocess
import os


def pinger(job_q, results_q):
    """
    Do Ping
    :param job_q:
    :param results_q:
    :return:
    """
    DEVNULL = open(os.devnull, 'w')
    while True:

        ip = job_q.get()

        if ip is None:
            break

        try:
            subprocess.check_call(['ping', '-c1', ip],
                                  stdout=DEVNULL)
            results_q.put(ip)
        except:
            pass


def get_my_ip():
    """
    Find my IP address
    :return:
    """
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    s.connect(("8.8.8.8", 80))
    ip = s.getsockname()[0]
    s.close()
    return ip


def map_network(pool_size=255):
    """
    Maps the network
    :param pool_size: amount of parallel ping processes
    :return: list of valid ip addresses
    """

    ip_list = list()

    # get my IP and compose a base like 192.168.1.xxx
    ip_parts = get_my_ip().split('.')
    base_ip = ip_parts[0] + '.' + ip_parts[1] + '.' + ip_parts[2] + '.'

    # prepare the jobs queue
    jobs = multiprocessing.Queue()
    results = multiprocessing.Queue()

    pool = [multiprocessing.Process(target=pinger, args=(jobs, results)) for i in range(pool_size)]

    for p in pool:
        p.start()

    # cue hte ping processes
    for i in range(1, 255):
        jobs.put(base_ip + '{0}'.format(i))

    for p in pool:
        jobs.put(None)

    for p in pool:
        p.join()

    # collect he results
    while not results.empty():
        ip = results.get()
        ip_list.append(ip)

    return ip_list


if __name__ == '__main__':

    print('Mapping...')
    lst = map_network()
    print(lst)

Mapping...
['192.168.7.1', '192.168.7.8', '192.168.7.24', '192.168.7.23', '192.168.7.45', '192.168.7.56', '192.168.7.67', '192.168.7.115', '192.168.7.132', '192.168.7.131', '192.168.7.99', '192.168.7.147', '192.168.7.127', '192.168.7.142', '192.168.7.73', '192.168.7.122', '192.168.7.125', '192.168.7.140', '192.168.7.124', '192.168.7.144', '192.168.7.149', '192.168.7.129', '192.168.7.130', '192.168.7.148', '192.168.7.168', '192.168.7.169', '192.168.7.167', '192.168.7.170', '192.168.7.137', '192.168.7.173', '192.168.7.165', '192.168.7.164', '192.168.7.219', '192.168.7.217', '192.168.7.212', '192.168.7.245', '192.168.7.251', '192.168.7.253']
